#**Face Generator Using GANS**

#**Application of GANs**
1. Generating fake faces
2. Generate Examples for Image Datasets3. Face Aging
4. Super Resolution
5. Image-to-Image Translation
6. Photos to Emojis
7.Text to image Translation
8. Generate Cartoon characters

In [ ]:
!unzip /kaggle/working/https:/www.kaggle.com/datasets/ashishjangra27/face-mask-12k-images-dataset/face-mask-12k-images-dataset.zip

In [ ]:
import re
import cv2
import numpy as np
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from tensorflow.keras import layers

##**Preprocess Dataset**

In [ ]:
img_size = 128
all_imgs = []

img_path = "/kaggle/working/Face Mask Dataset/Train/WithoutMask"
files = sorted(os.listdir(img_path))

for i in tqdm(files):
    img = cv2.imread(img_path + "/" + i, 1)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (img_size, img_size))
    img = (img - 127.5)  / 127.5
    img = img.astype(float)
    all_imgs.append(keras.utils.img_to_array(img))

In [ ]:
def plot_img(ran):
    plt.figure(figsize=(10,10))
    plt.title("Real images", fontsize = 35)

    for i in range(ran * ran):
        plt.subplot(ran, ran, i+1)
        plt.imshow(all_imgs[i] * 0.5 + 0.5)
        plt.xticks([])
        plt.yticks([])
    plt.show()


plot_img(6)

##**Convert Dataset to Tensor Slices**

In [ ]:
batch_size = 32
buffer_size = 60000
dataset = tf.data.Dataset.from_tensor_slices(all_imgs).shuffle(buffer_size).batch(batch_size)

##**Build Generator Model**

In [ ]:
latent_dim = 100


def Generator():
    model = keras.Sequential()
    model.add(keras.Input(shape = (latent_dim,)))
    model.add(layers.Dense(128*128*3, use_bias=False))
    model.add(layers.Reshape((128,128,3)))
    # downsampling
    model.add(layers.Conv2D(128,4, strides=1, padding='same',kernel_initializer='he_normal', use_bias=False))
    model.add(layers.Conv2D(128,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2D(256,4, strides=1, padding='same',kernel_initializer='he_normal', use_bias=False))
    model.add(layers.Conv2D(256,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2DTranspose(512, 4, strides=1,padding='same',kernel_initializer='he_normal',use_bias=False))
    model.add(layers.Conv2D(512,4, strides=2, padding='same',kernel_initializer='he_normal', use_bias=False))

    model.add(layers.LeakyReLU())
    #upsampling
    model.add(layers.Conv2DTranspose(512, 4, strides=1,padding='same',kernel_initializer='he_normal',use_bias=False))
    model.add(layers.Conv2DTranspose(512, 4, strides=2,padding='same',kernel_initializer='he_normal',use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Conv2DTranspose(256, 4, strides=1,padding='same',kernel_initializer='he_normal',use_bias=False))
    model.add(layers.Conv2DTranspose(256, 4, strides=2,padding='same',kernel_initializer='he_normal',use_bias=False))
    model.add(layers.BatchNormalization())

    model.add(layers.Conv2DTranspose(128, 4, strides=2,padding='same',kernel_initializer='he_normal',use_bias=False))
    model.add(layers.Conv2DTranspose(128, 4, strides=1,padding='same',kernel_initializer='he_normal',use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2DTranspose(3,4,strides = 1, padding = 'same',activation = 'tanh'))



    return model

In [ ]:
generator = Generator()
generator.summary()

In [ ]:
noise = np.random.normal(-1,1,(1,100))
img = generator(noise)
plt.imshow(img[0,:,:,0])
plt.show()

##**Build Discriminator Model**

In [ ]:
def Discriminator():
    model = keras.Sequential()
    model.add(keras.Input(shape = (img_size, img_size, 3)))
    model.add(layers.Conv2D(128, 4, strides = 2, padding = "same", kernel_initializer = "he_normal", use_bias = False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(128, 4, strides = 2, padding = "same", kernel_initializer = "he_normal", use_bias = False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, 4, strides = 2, padding = "same", kernel_initializer = "he_normal", use_bias = False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(256, 4, strides = 2, padding = "same", kernel_initializer = "he_normal", use_bias = False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Conv2D(512, 4, strides = 2, padding = "same", kernel_initializer = "he_normal", use_bias = False))
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))

    model.add(layers.Flatten())
    model.add(layers.Dense(1, activation = "sigmoid"))

    return model

In [ ]:
discriminator = Discriminator()
discriminator.summary()

In [ ]:
gen_optimizer = keras.optimizers.SGD(
    learning_rate = 0.0001,
    clipvalue = 1.0,
    weight_decay = 1e-4)

disc_optimizer = keras.optimizers.SGD(
    learning_rate = 0.0001,
    clipvalue = 1.0,
    weight_decay = 1e-4)

cr_en_loss = keras.losses.BinaryCrossentropy(from_logits = True)

In [ ]:
def generator_loss(fake_output):
    return cr_en_loss(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = cr_en_loss(tf.ones_like(real_output), real_output)
    fake_loss = cr_en_loss(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

##**Build Training Steps**

In [ ]:
def training_steps(images):
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_img = generator(noise)

        fake_output = discriminator(generated_img)
        real_output = discriminator(images)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradient_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradient_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    gen_optimizer.apply_gradients(zip(gradient_of_generator,generator.trainable_variables))
    disc_optimizer.apply_gradients(zip(gradient_of_discriminator, discriminator.trainable_variables))

    loss = {'gen loss':gen_loss,
           'disc loss': disc_loss}

    return loss

In [ ]:
import time
def train(epochs,dataset):

    for epoch in range(epochs):
        start = time.time()
        print("\nEpoch : {}".format(epoch + 1))
        for images in tqdm(dataset):
            loss = training_steps(images)
        print(" Time:{}".format(np.round(time.time() - start),2))
        print("Generator Loss: {} Discriminator Loss: {}".format(loss['gen loss'],loss['disc loss']))

##**Train the Dataset**
if ~100 epochs use to get better results 

In [ ]:
train(40, dataset)

In [ ]:
def plot_generated_images(square = 5, epochs = 0):
    
    
  plt.figure(figsize = (10,10))
  for i in range(square * square):
    if epochs != 0:    
        if(i == square //2):
            plt.title("Generated Image at Epoch:{}\n".format(epochs), fontsize = 32, color = 'black')
    plt.subplot(square, square, i+1)
    noise = np.random.normal(0,1,(1,latent_dim))
    img = generator(noise)
    plt.imshow(np.clip((img[0,...]+1)/2, 0, 1))
    
    plt.xticks([])
    plt.yticks([])
    plt.grid()

In [ ]:
plot_generated_images(4)

At least 100 epochs Train to get better  results 